Prepare simplified street graph
===============================
Downloads OSM data and prepares a simplified street graph. The street graph gets saved as a GeoPackage file and can be loaded later.

In [ ]:
import geopandas as gpd
import networkx as nx
import snman

# we use a customized fork of osmnx
# it may be added to osmnx as a pull request in the future
from snman import osmnx_customized as oxc

# =====================================================================================
# CONSTANTS
# =====================================================================================

PERIMETER = 'zrh_north-west'
# Set SAVE_TO_DEBUG = True for saving the cached network into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True
INTERSECTION_TOLERANCE = 10

# set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data/'
inputs_path = data_directory + 'inputs/'
if SAVE_TO_DEBUG:
    process_path = data_directory + 'process/' + '_debug' + '/'
else:
    process_path = data_directory + 'process/' + PERIMETER + '/'

oxc.settings.useful_tags_way = snman.constants.OSM_TAGS

In [ ]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load perimeters')
perimeters = snman.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

print('Get data from OSM server')
# At this step, simplification means only removing degree=2 edges
G = oxc.graph_from_polygon(
    # set the perimeter here
    perimeters.to_crs(4326).loc[PERIMETER]['geometry'],
    custom_filter=snman.constants.OSM_FILTER,
    simplify=True, simplify_strict=False, retain_all=True, one_edge_per_direction=False
)

print('Prepare graph')
snman.prepare_graph(G)

print('Convert CRS of street graph to 2056')
snman.convert_crs_of_street_graph(G, snman.constants.CRS)

print('Load regions')
# polygons that define local parameters for the simplification algorithms
regions_gdf = snman.load_regions(inputs_path + 'regions/regions.shp', default_tolerance=10, street_graph=G)

print('Load manual intersections')
# polygons used to override the automatically detected intersections in some situations
given_intersections_gdf = snman.load_intersections(
    inputs_path + 'intersection_polygons/intersection_polygons.shp'
)

print('Load reorganization regions')
# polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

In [ ]:
# =====================================================================================
# EXPORT RAW NETWORK
# =====================================================================================

if 1:
    print('Save raw street graph')
    snman.export_street_graph(G, process_path + 'raw_edges.gpkg', process_path + 'raw_nodes.gpkg')

In [ ]:
# =====================================================================================
# ENRICH RAW NETWORK
# =====================================================================================

if 1:
    print('Load sensors and assign them to edges in the raw street graph')
    sensors_df = snman.io.load_sensors(inputs_path + 'sensors/sensors.csv')
    snman.enrichment.match_sensors(G, sensors_df)

In [ ]:
# =====================================================================================
# CONSOLIDATE INTERSECTIONS
# =====================================================================================

print('Detect intersections')
intersections_gdf = snman.simplification.merge_nodes_geometric(
    G, INTERSECTION_TOLERANCE,
    given_intersections_gdf=given_intersections_gdf,
    regions_gdf=regions_gdf
)

if 1:
    # must be run a few times for including buffers of newly added nodes
    for i in range(3):
        print('Split through edges in intersections')
        intersections = snman.split_through_edges_in_intersections(G, intersections_gdf)

        print('Add layers to nodes')
        snman.graph_tools._add_layers_to_nodes(G)

        print('Update precalculated attributes')
        snman.update_precalculated_attributes(G)

        print('Detect intersections')
        intersections_gdf = snman.simplification.merge_nodes_geometric(
            G, INTERSECTION_TOLERANCE,
            given_intersections_gdf=given_intersections_gdf,
            regions_gdf=regions_gdf
        )

        print('Add connections between components in intersections')
        snman.connect_components_in_intersections(G, intersections_gdf, separate_layers=True)

    print('Save intersection geometries into a file')
    snman.export_gdf(intersections_gdf, process_path + 'intersections_polygons.gpkg', columns=['geometry'])

if 1:
    print('Consolidate intersections')
    G = snman.simplification.consolidate_intersections(
        G, intersections_gdf,
        reconnect_edges=True
    )

In [ ]:
# =====================================================================================
# ENRICH AND ADJUST GRAPH
# =====================================================================================

if 1:
    print('Generate lanes')
    # interpreting the OSM tags into a collection of lanes on each edge
    snman.generate_lanes(G)

if 1:
    print('Organize edge directions, enforce direction from lower to higher node id')
    snman.organize_edge_directions(G)

if 1:
    print('Convert into an undirected graph')
    G = oxc.utils_graph.get_undirected(G)

if 1:
    print('Identify hierarchy')
    # split the edges into hierarchy categories, such as main roads, local roads, etc.
    snman.add_hierarchy(G)

In [ ]:
# =====================================================================================
# CONSOLIDATE PARALLEL AND CONSECUTIVE EDGES
# =====================================================================================

if 1:
    print('Merge parallel and consecutive edges, repeat a few times')
    for i in range(5):
        snman.merge_parallel_edges(G)
        snman.merge_consecutive_edges(G)
        pass

if 1:
    print('Simplify edge geometries')
    for id, edge in G.edges.items():
        edge['geometry'] = edge['geometry'].simplify(25, preserve_topology=False)

if 1:
    print('Add lane stats to edges')
    # how many lanes, how wide, etc.
    snman.generate_lane_stats(G)

In [ ]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 0:
    #TODO: use mapmatching for better performance and accuracy
    #TODO: add route direction for one-way sections
    print('Add public transport')
    pt_network = snman.import_geofile_to_gdf(inputs_path + "public_transit/ZVV_LINIEN_GEN_L.shp")
    snman.match_pt(G, pt_network)

if 1:
    print('Update OSM tags')
    # to match the simplified and merged edges
    snman.update_osm_tags(G)

if 1:
    print('Add elevation')
    spn = oxc.stats.count_streets_per_node(G, nodes=G.nodes)
    nx.set_node_attributes(G, values=spn, name="street_count")
    G = oxc.elevation.add_node_elevations_raster(G, inputs_path + 'ch_dhm_25/2056/ch_dhm_2056.tif', cpus=1)
    G = oxc.elevation.add_edge_grades(G, add_absolute=False)

if 0:
    print('Add traffic counts')
    source = gpd.read_file(inputs_path + 'traffic_volumes/npvm_2017_filtered.gpkg').to_crs(2056)
    source['fid'] = source.index
    # Remove links with zero traffic (otherwise they will distort the averages on the matched links)
    source = source[source['DTV_ALLE'] > 0]
    snman.enrichment.match_linestrings(G, source, [
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_avg',         'agg': 'avg' },
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_max',         'agg': 'max' },
        {'source_column': 'FROMNODENO', 'target_column': 'npvm_fromnodeno', 'agg': 'list'},
        {'source_column': 'TONODENO',   'target_column': 'npvm_tonodeno',   'agg': 'list'}
        ]
    )

In [ ]:
# =====================================================================================
# VARIA
# =====================================================================================

if 1:
    print('Keep only the largest connected component')
    # remove edges that are not connected to the rest of the network
    snman.graph_tools.add_connected_component_ids(G)
    G = snman.graph_tools.keep_only_the_largest_connected_component(G)

In [ ]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export basic street graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.export_street_graph(G, process_path + 'edges_all_attributes.gpkg', process_path + 'nodes_all_attributes.gpkg')

if 1:
    print('Export lane geometries')
    # each lane has an own geometry and with as an attribute, for visualization purposes
    snman.export_street_graph_with_lanes(G, 'ln_desc', process_path + 'edges_lanes.shp', scaling=2)